In [1]:
import tkinter as tk
from tkinter.ttk import *
import numpy as np
import time


In [2]:
class Cell:
    def __init__(self,grid,row,col,state):
        self.grid = grid
        self.row = row
        self.col = col
        self.state = state
        self.neighbors = []
    def appear(self):
        self.state = 1
    def dead(self):
        self.state = 0
    def update_state(self):
        if self.state == 1:
            if (self.grid.counts[self.row][self.col]>3) or (self.grid.counts[self.row][self.col]<2):
                self.dead()
        else:
            if (self.grid.counts[self.row][self.col]==3):
                self.appear()

                
class Grid:
    def __init__(self,n_rows,n_cols):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.cells = np.array([[Cell(self,row,col,0) for col in range(0,n_cols+2)] for row in range(0,n_rows+2)])
        # add 2 cols and rows for count_align_neighbors
        
        self.get_neighbors()
        self.count_align_neighbors()
        self.counts = self.count_align_neighbors()
    
    def get_neighbors(self):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                neighbors = []
                for i in range(-1,2):
                    for j in range(-1,2):
                        neighbors.append(self.cells[row+i][col+j])
                del neighbors[4]                 #minus itself
                self.cells[row][col].neighbors = neighbors
                
    def count_align_neighbors(self):
        self.counts = [[0 for col in range(0,n_cols+2)] for row in range(0,n_rows+2)]
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                neighbors_state = 0
                
                for neighbor in self.cells[row][col].neighbors:
                    neighbors_state += neighbor.state
                    
                self.counts[row][col] = neighbors_state 
        return self.counts
    def update_grid_state(self):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                self.cells[row][col].update_state()
class Click:  
    def Update_frame(grid,n_rows,n_cols,cell_buttons):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                if grid.cells[row][col].state==1:
                    cell_buttons[row-1][col-1].config(bg='yellow')
                else:
                    cell_buttons[row-1][col-1].config(bg='gray')
    def Choose_cell_button(grid,cell_buttons,row, col):
        button = cell_buttons[row][col]
        button.config(bg="#14f500")
        grid.cells[row+1][col+1].appear()
        choose_cell_label = tk.Label(root,text=f"Bạn đã chọn ô hàng {row+1}, cột {col+1}",bg='skyblue')
        choose_cell_label.place(x=50,y=380)
        next_grid_button = tk.Button(root,text='Start',bg='red',command=lambda:Click.Start(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)
    def Start(grid,cell_buttons):
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        
        next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)
        next_100_grid_button = tk.Button(root,text='Run',command=lambda:Click.Run(grid,cell_buttons))
        next_100_grid_button.place(x=350,y=380)
        restart_button = tk.Button(root,text='Restart',command=lambda:Click.Restart(grid,cell_buttons))
        restart_button.place(x=450,y=380)
    def Next(grid,cell_buttons):
        grid.count_align_neighbors()
        grid.update_grid_state()
        grid.count_align_neighbors()
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        Move_frame.update_bounding(grid,cell_buttons)
    def Stop_run():
        global result
        result = False
        stop_run_label = tk.Label(root,text=f"Bạn đã dừng chuyển động !!!!",bg='#fc8403')
        stop_run_label.place(x=50,y=380)
    def Run(grid,cell_buttons): 
        global result
        result = True
        running_label = tk.Label(root,text=f"Đang thực hiện chuyển động",bg='#03fc5e')
        running_label.place(x=50,y=380)
        while result:
            next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
            next_grid_button.invoke()
            root.update()
            Move_frame.update_bounding(grid,cell_buttons)
            time.sleep(0.5)
            stop_loop_button = tk.Button(root,text='Stop',bg='red',command=lambda:Click.Stop_run())
            stop_loop_button.place(x=520,y=380)
    def Restart(grid,cell_buttons):
        for row in range(1,n_rows+1):
            for col in range(1,n_cols+1):
                grid.cells[row][col].dead()
        Click.Update_frame(grid,n_rows,n_cols,cell_buttons)
        next_grid_button = tk.Button(root,text='Next',command=lambda:Click.Next(grid,cell_buttons))
        next_grid_button.place(x=300,y=380)        
class Move_frame:
    def Update_state_after_move(grid,state_arr):   #after move the frame
        for i in range(n_rows+2):
            for j in range(n_cols+2):
                grid.cells[i][j].state = state_arr[i][j]
    def Up_frame(grid,cell_buttons):
        num_move = n_rows//2
        state_arr = [[grid.cells[i][j].state for j in range(0,n_cols+2)] for i in range(0,n_rows+2)]
        
        new_rows = [[0 for col in range(0,n_cols+2)] for row in range(num_move)]
        added_rows_grid = np.vstack((new_rows,state_arr))
        state_arr = added_rows_grid[:n_rows+2]
        Move_frame.Update_state_after_move(grid,state_arr)
        Click.Start(grid,cell_buttons)
    def Down_frame(grid,cell_buttons):
        num_move = n_rows//2
        state_arr = [[grid.cells[i][j].state for j in range(0,n_cols+2)] for i in range(0,n_rows+2)]
        
        new_rows = [[0 for col in range(0,n_cols+2)] for row in range(num_move)]
        added_rows_grid = np.vstack((state_arr,new_rows))
        state_arr = added_rows_grid[num_move:]
        Move_frame.Update_state_after_move(grid,state_arr)
        Click.Start(grid,cell_buttons)
    def Left_frame(grid,cell_buttons):
        num_move = n_cols//2
        state_arr = [[grid.cells[i][j].state for j in range(0,n_cols+2)] for i in range(0,n_rows+2)]
        
        new_rows = [[0 for col in range(num_move)] for row in range(0,n_rows+2)]
        added_rows_grid = np.hstack((new_rows,state_arr))
        state_arr = added_rows_grid[:][:n_cols+2]
        Move_frame.Update_state_after_move(grid,state_arr)
        Click.Start(grid,cell_buttons)
        
    def Right_frame(grid,cell_buttons):
        num_move = n_cols//2
        state_arr = [[grid.cells[i][j].state for j in range(0,n_cols+2)] for i in range(0,n_rows+2)]

        new_rows = [[0 for col in range(num_move)] for row in range(0,n_rows+2)]
        added_rows_grid = np.hstack((state_arr,new_rows))
        state_arr = added_rows_grid[:,num_move:]
        Move_frame.Update_state_after_move(grid,state_arr)
        Click.Start(grid,cell_buttons)    
    def warning_window(grid,type_warning,cell_buttons):
        message_box = tk.Toplevel(root)
        message_box.title("Cảnh báo") 
        warning_label = tk.Label(message_box, text="Ô sống đã vượt biên, bạn có muốn mở rộng biên không?")
        warning_label.pack()
        if type_warning =='up':
            update_button = tk.Button(message_box, text="Mở rộng lên", command=lambda:[Move_frame.Up_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='down':
            update_button = tk.Button(message_box, text="Mở rộng xuống", command=lambda:[Move_frame.Down_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='left':
            update_button = tk.Button(message_box, text="Mở rộng trái", command=lambda:[Move_frame.Left_frame(grid,cell_buttons),message_box.destroy()])
        if type_warning =='right':
            update_button = tk.Button(message_box, text="Mở rộng phải", command=lambda:[Move_frame.Right_frame(grid,cell_buttons),message_box.destroy()])        
        update_button.pack()
        message_box.wait_window()        
    def update_bounding(grid,cell_buttons):
        for i in range(1,n_cols+1):
            if grid.cells[1][i].state == 1:
                Move_frame.warning_window(grid,'up',cell_buttons)
            if grid.cells[n_rows][i].state ==1:
                Move_frame.warning_window(grid,'down',cell_buttons)
        for i in range(1,n_rows+1):
            if grid.cells[i][1].state == 1:
                Move_frame.warning_window(grid,'left',cell_buttons)
            if grid.cells[i][n_cols].state == 1:
                Move_frame.warning_window(grid,'right',cell_buttons)
        
class create_click:
    def create_grid_button():
        grid = Grid(n_rows,n_cols)
        overlay = tk.Canvas(root, bg="gray", highlightthickness=0)
        overlay.place(relwidth=1, relheight=1)
        for i in range(n_rows + 1):
            root.rowconfigure(i, weight=1)
        for j in range(n_cols + 1):
            root.columnconfigure(j, weight=1)
        cell_buttons = [[None] * n_cols for _ in range(n_rows)]
        for i in range(n_rows):
            for j in range(n_cols):
                cell_button = tk.Button(root,command=lambda row=i, col=j: Click.Choose_cell_button(grid,cell_buttons,row, col))
                cell_button.grid(row=i, column=j,sticky="nsew")
                cell_buttons[i][j] = cell_button
def get_num():
    try:
        global n_rows,n_cols
        n_rows = int(n_rows_entry.get())
        n_cols = int(n_cols_entry.get())
        if isinstance(n_rows,int) & isinstance(n_cols,int) & (n_rows>=5) & (n_cols>=5) & (n_rows<20) & (n_cols<20):
            get_num_checked_label = tk.Label(root,text="Số thỏa mãn,chơi thôi",fg='green')
            get_num_checked_label.place(x=150,y=130)
            start_btn = Button(root,text="Chơi",command=create_click.create_grid_button)
            start_btn.place(x=250,y=300)
        else:
            get_num_error_label = tk.Label(root,text="Nhập số không thỏa mãn\n Hãy nhập số nguyên, và lớn hơn 4 và không vượt quá 20 cho trải nghiệm chơi tốt nhất",
                                       fg='red')
            get_num_error_label.place(x=50,y=160)
    except:
        get_num_error_label = tk.Label(root,text="Nhập số không thỏa mãn\n Hãy nhập số nguyên, và lớn hơn 4 và không vượt quá 20 cho trải nghiệm chơi tốt nhất",
                                       fg='red')
        get_num_error_label.place(x=50,y=160)

In [3]:
root = tk.Tk()
root.geometry('600x400')
root.resizable(0,0)
root.title('Conway Game')
root.resizable(width=False, height=False)


# Label
n_rows_entry_label = tk.Label(root,text="Nhập số hàng:")
n_rows_entry_label.place(x=50,y=50)

n_cols_entry_label = tk.Label(root,text="Nhập số cột:")
n_cols_entry_label.place(x=50,y=100)

# Entry
n_rows_entry = tk.Entry(root)
n_rows_entry.place(x=150,y=50,width=20)

n_cols_entry = tk.Entry(root)
n_cols_entry.place(x=150,y=100,width=20)

# Button
start_button = tk.Button(root,text='Lấy số',command=get_num)
start_button.place(x=100,y=130)


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_12760\1941254729.py", line 78, in <lambda>
    next_100_grid_button = tk.Button(root,text='Run',command=lambda:Click.Run(grid,cell_buttons))
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_12760\1941254729.py", line 102, in Run
    Move_frame.update_bounding(grid,cell_buttons)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_12760\1941254729.py", line 175, in update_bounding
    Move_frame.warning_window(grid,'down',cell_buttons)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_12760\1941254729.py", line 156, in warning_window
    message_box = tk.Toplevel(root)
  File "C:\Users\Admin\anaconda3\lib\tkinter\__init__.py", line 2621, in __init__
    BaseWidget.__init__(self, master, 'toplevel', cnf, {}, extra)
  File "C:\Users\Admin\anaconda3\lib\tkinter\_